# Xbox Datenaufbereitung und Lineare Regression

In diesem Notebook werden die Xbox-Daten verarbeitet und ein Modell für die Vorhersage von Konsolenpreisen mithilfe von Linearer Regression erstellt.

## 1. Daten laden und anzeigen
Hier werden die Daten aus der CSV-Datei geladen und die ersten Zeilen angezeigt.

In [53]:

import os
import pandas as pd

# Datei mit den Daten
data_file = "/Users/Hannes/Samsaru/asgoodasnew_products.csv"

# Daten laden
if os.path.exists(data_file):
    df = pd.read_csv(data_file)
    display(df.head())
else:
    print(f"Die Datei {data_file} existiert nicht!")


,scrape_date,title,price,brand,category,variant
0,2025-01-08 14:22:15,Microsoft Xbox Series X - 1TB schwarz,589.0,Microsoft,Konsolen,neu
1,2025-01-08 14:22:15,Microsoft Xbox Series X - 1TB schwarz,569.0,Microsoft,Konsolen,wie neu
2,2025-01-08 14:22:15,Microsoft Xbox Series X - 1TB schwarz,549.0,Microsoft,Konsolen,sehr gut
3,2025-01-08 14:22:15,Microsoft Xbox Series X - 1TB schwarz,539.0,Microsoft,Konsolen,gut
4,2025-01-08 14:22:15,Microsoft Xbox Series S - 512GB weiß,289.0,Microsoft,Konsolen,neu


## 2. Datenverarbeitung
Hier wird der Titel in `model` und `storage_capacity` aufgeteilt. Außerdem werden neue Features wie `variant_score` und `price_deviation` berechnet.

In [54]:

# Preis in numerische Werte umwandeln
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df.dropna(subset=['price'], inplace=True)

# Titel aufteilen in Modell und Speicherkapazität
df[['model', 'storage_capacity']] = df['title'].str.extract(r'(Microsoft Xbox [^–]+) - ([^ ]+)')
df.drop(columns=['title'], inplace=True)

# Zustand in numerische Werte kodieren
df['variant_score'] = df['variant'].map({'neu': 1, 'wie neu': 0.8, 'sehr gut': 0.6, 'gut': 0.4})

# Durchschnittspreis nach Modell berechnen
df['avg_price_by_title'] = df.groupby('model')['price'].transform('mean')
df['price_deviation'] = df['price'] - df['avg_price_by_title']

# Marke und Kategorie kodieren
df['brand'] = df['brand'].astype('category').cat.codes
df['category'] = df['category'].astype('category').cat.codes

# Speichergröße in numerische Werte umwandeln
def parse_storage_capacity(value):
    if 'TB' in value:
        return int(float(value.replace('TB', '').strip()) * 1024)  # TB zu GB umwandeln
    elif 'GB' in value:
        return int(value.replace('GB', '').strip())
    return None

df['storage_capacity'] = df['storage_capacity'].apply(parse_storage_capacity)

# Entferne Zeilen mit ungültiger Speichergröße
df.dropna(subset=['storage_capacity'], inplace=True)

# Verarbeitete Daten anzeigen
display(df)



,scrape_date,price,brand,category,variant,model,storage_capacity,variant_score,avg_price_by_title,price_deviation
0,2025-01-08 14:22:15,589.00,0,0,neu,Microsoft Xbox Series X,1024,1.0,494.831538,94.168462
1,2025-01-08 14:22:15,569.00,0,0,wie neu,Microsoft Xbox Series X,1024,0.8,494.831538,74.168462
2,2025-01-08 14:22:15,549.00,0,0,sehr gut,Microsoft Xbox Series X,1024,0.6,494.831538,54.168462
3,2025-01-08 14:22:15,539.00,0,0,gut,Microsoft Xbox Series X,1024,0.4,494.831538,44.168462
4,2025-01-08 14:22:15,289.00,0,0,neu,Microsoft Xbox Series S,512,1.0,358.390345,-69.390345
5,2025-01-08 14:22:15,269.00,0,0,wie neu,Microsoft Xbox Series S,512,0.8,358.390345,-89.390345
6,2025-01-08 14:22:15,249.00,0,0,sehr gut,Microsoft Xbox Series S,512,0.6,358.390345,-109.390345
7,2025-01-08 14:22:15,219.00,0,0,gut,Microsoft Xbox Series S,512,0.4,358.390345,-139.390345
8,2025-01-08 14:22:15,430.41,0,0,neu,Microsoft Xbox Series S,1024,1.0,358.390345,72.019655
9,2025-01-08 14:22:15,388.58,0,1,neu,Microsoft Xbox One S,500,1.0,264.592500,123.987500


## 3. Lineare Regression
Hier wird ein Modell zur Vorhersage der Konsolenpreise mit Linearer Regression trainiert.

In [55]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

# Feature-Auswahl
features = ['storage_capacity', 'variant_score', 'price_deviation', 'brand', 'category']
X = df[features]
y = df['price']

# Daten aufteilen in Training und Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modell initialisieren und trainieren (mit Regularisierung)
ridge_model = Ridge(alpha=3.0)  # alpha ist der Regularisierungsparameter (kann angepasst werden)
ridge_model.fit(X_train, y_train)

# Vorhersagen
y_pred = ridge_model.predict(X_test)

# Ergebnisse bewerten
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

# Ergebnisse ausgeben
print(f"R²-Wert (Ridge Regression): {r2:.2f}")
print(f"Mean Squared Error (Ridge Regression): {mse:.2f}")

# Beispiel: Zeige die Feature-Namen und R²-Wert
if features:
    print("Verwendete Features:", features)


R²-Wert (Ridge Regression): 0.73
Mean Squared Error (Ridge Regression): 3549.81
Verwendete Features: ['storage_capacity', 'variant_score', 'price_deviation', 'brand', 'category']


## 4. Speichern des Modells
Das trainierte Modell wird gespeichert, damit es später wiederverwendet werden kann.

In [56]:

import pickle

model_file = "linear_model.pkl"

# Modell speichern
try:
    with open(model_file, 'wb') as f:
        pickle.dump({"model": model, "features": features, "r2": r2}, f)
    print(f"Modell erfolgreich in {os.path.abspath(model_file)} gespeichert.")
except Exception as e:
    print(f"Fehler beim Speichern des Modells: {e}")


Modell erfolgreich in /Users/Hannes/Desktop/linear_model.pkl gespeichert.


## 5. Fazit
Das Modell wurde erfolgreich trainiert und gespeichert. Es kann verwendet werden, um Konsolenpreise basierend auf ihren Merkmalen vorherzusagen.